<a href="https://colab.research.google.com/github/bugatha1/TF-DistributtedTraining/blob/main/SimpleTFDistributed_Training_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf

In [9]:
from tensorflow.keras import datasets, layers, models
import numpy as np
import os
from datetime import datetime

In [10]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [11]:
print('Number of devices: {}'.format(
 strategy.num_replicas_in_sync))

Number of devices: 1


In [13]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [14]:
# Plain-text name in alphabetical order. See https://www.cs.toronto.edu/~kriz/cifar.html
CLASS_NAMES = ['airplane', 'automobile', 'bird', 'cat', 
               'deer','dog', 'frog', 'horse', 'ship', 'truck']

In [15]:
validation_dataset = tf.data.Dataset.from_tensor_slices(
(test_images[:500], 
 test_labels[:500]))

test_dataset = tf.data.Dataset.from_tensor_slices(
(test_images[500:], 
 test_labels[500:]))

train_dataset = tf.data.Dataset.from_tensor_slices(
(train_images,
 train_labels))

In [16]:
train_dataset_size = len(list(train_dataset.as_numpy_iterator()))
print('Training data sample size: ', train_dataset_size)

validation_dataset_size = len(list(validation_dataset.
as_numpy_iterator()))
print('Validation data sample size: ', validation_dataset_size)

test_dataset_size = len(list(test_dataset.as_numpy_iterator()))
print('Test data sample size: ', test_dataset_size)

Training data sample size:  50000
Validation data sample size:  500
Test data sample size:  9500


In [17]:
TRAIN_BATCH_SIZE = 128
train_dataset = train_dataset.shuffle(50000).batch(TRAIN_BATCH_SIZE, drop_remainder=True)

validation_dataset = validation_dataset.batch(validation_dataset_size)
test_dataset = test_dataset.batch(test_dataset_size)

In [18]:
STEPS_PER_EPOCH = train_dataset_size // TRAIN_BATCH_SIZE
VALIDATION_STEPS = 1

In [19]:
with strategy.scope():
  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3),activation='relu', name = 'conv_1',kernel_initializer='glorot_uniform',padding='same', input_shape = (32,32,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3),activation='relu', name = 'conv_2',kernel_initializer='glorot_uniform', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(name = 'flat_1'),
    tf.keras.layers.Dense(256, activation='relu',kernel_initializer='glorot_uniform', name = 'dense_64'),
    tf.keras.layers.Dense(10, activation='softmax',name = 'custom_class')
  ])
  model.build([None, 32, 32, 3])

  model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [20]:
MODEL_NAME = 'myCIFAR10-{}'.format(datetime.now().strftime("%Y%m%d-%H%M%S"))

checkpoint_dir = './' + MODEL_NAME
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt-{epoch}")
print(checkpoint_prefix)

./myCIFAR10-20220622-113643/ckpt-{epoch}


In [21]:
myCheckPoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=True,
    save_best_only = True)

In [22]:
myCallbacks = [
    myCheckPoint
]

In [23]:
hist = model.fit(
    train_dataset,
    epochs=12,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_dataset,
    validation_steps=VALIDATION_STEPS,
    callbacks=myCallbacks).history

Epoch 1/12
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
390/390 [==============================] - 18s 11ms/step - loss: 1.4769 - accuracy: 0.4716 - val_loss: 1.2325 - val_accuracy: 0.5640
Epoch 2/12
390/390 [==============================] - 3s 7ms/step - loss: 1.0922 - accuracy: 0.6171 - val_loss: 1.0253 - val_accuracy: 0.6380
Epoch 3/12
390/390 [==============================] - 3s 7ms/step - loss: 0.9455 - accuracy: 0.6710 - val_loss: 0.9519 - val_accuracy: 0.6840
Epoch 4/12
390/390 [==============================] - 3s 7ms/step - loss: 0.8547 - accuracy: 0.7035 - val_loss: 0.9804 - val_accuracy: 0.6600
Epoch 5/12
390/390 [==============================] - 3s 7ms/step - loss: 0.7809 - accuracy: 0.7296 - val_loss: 0.8870 - val_accuracy: 0.6960
Epoch 6/12
390/390 [==============================] - 3s 7ms/step - loss: 0.7083 - accuracy: 0.7548 - val_loss: 0.9352 - val_accuracy: 0.6800
Epoch 7/12
390/39

In [24]:
hist['val_accuracy']

[0.5640000104904175,
 0.6380000114440918,
 0.6840000152587891,
 0.6600000262260437,
 0.6959999799728394,
 0.6800000071525574,
 0.6859999895095825,
 0.6980000138282776,
 0.6940000057220459,
 0.7059999704360962,
 0.7200000286102295,
 0.7059999704360962]

In [25]:
max_value = max(hist['val_accuracy'])
max_index = hist['val_accuracy'].index(max_value)
print('Best epoch: ', max_index + 1)

Best epoch:  11


In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./myCIFAR10-20220622-113643/ckpt-11'

In [27]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [28]:
eval_loss, eval_acc = model.evaluate(test_dataset)
print('Eval loss: {}, Eval Accuracy: {}'.format(
eval_loss, eval_acc))

1/1 [==============================] - 2s 2s/step - loss: 0.9070 - accuracy: 0.7243
Eval loss: 0.9070010185241699, Eval Accuracy: 0.7243157625198364
